# Assignment 7: CNNs & RNNs

In [130]:
# enter your name and UFL email address
name = 'Kamal Sai Raj Kuncha'
email = 'k.kuncha@ufl.edu'

In [131]:
if name == 'enter your name' or email == 'enter your email':
    assert False, 'Enter your name & email first!'
else:
    print('Assignment 7 -- name: {}, email: {}\n'.format(name, email))
    
    # Load packages we need
    import sys
    import os
    import time

    import numpy as np
    import sklearn
    
    # we'll use tensorflow and keras for neural networks
    import tensorflow as tf
    import tensorflow.keras as keras
    
    # import layers we may use
    from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, Dropout, Activation

    # import callbacks we may use
    from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
    
    # Load the TensorBoard notebook extension
    #%load_ext tensorboard

    from matplotlib import pyplot as plt
    plt.rcParams.update({'font.size': 16})

    # Let's check our software versions
    print('### Python version: ' + __import__('sys').version)
    print('### NumPy version: ' + np.__version__)
    print('### Scikit-learn version: ' + sklearn.__version__)
    print('### Tensorflow version: ' + tf.__version__)
    print('### TF Keras version: ' + keras.__version__)
    print('------------')


    # load our packages / code
    sys.path.insert(1, '../common/')
    import utils
    import plots

Assignment 7 -- name: Kamal Sai Raj Kuncha, email: k.kuncha@ufl.edu

### Python version: 3.6.5 (v3.6.5:f59c0932b4, Mar 28 2018, 17:00:18) [MSC v.1900 64 bit (AMD64)]
### NumPy version: 1.19.5
### Scikit-learn version: 0.24.0
### Tensorflow version: 2.4.0
### TF Keras version: 2.4.0
------------


In [132]:
# global parameters to control behavior of the pre-processing, ML, analysis, etc.
seed = 42

# deterministic seed for reproducibility
np.random.seed(seed)
tf.random.set_seed(seed)

prop_vec = [24, 2, 2]

## [Task 1] (20 points) Loading and Processing CIFAR-10

### [Task 1a] (20 points) Complete the implementation of load_preprocess_cifar10(). Make sure you correctly implement all of the cases.

In [133]:
from tensorflow.keras.datasets import cifar10
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
# refer to: https://www.tensorflow.org/api_docs/python/tf/keras/datasets/cifar10/load_data
# and to https://www.cs.toronto.edu/~kriz/cifar.html
def load_preprocess_cifar10(onehot=True, minmax_normalize=True):
    
    labels = np.array(['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'])
    
    ### Load and preprocess the cifar10 data, then split it into train, test, validation
    ### The shapes of train_x, test_x, val_x should be: (50000, 32, 32, 3), (5000, 32, 32, 3), (5000, 32, 32, 3)
    ### If onehot=True you need to one hot encode the labels (y vector)
    ### If minmax_normalize=True you need to minmax normalize the pixel values to be in the range [0,1]
    ###* put your code here (~10-20 lines) *###
    #print(keras.backend.image_data_format())#'channels_last'
    train,testval=cifar10.load_data()
    train_x, train_y = train
    testval_x, testval_y = testval
    
    if minmax_normalize:
        train_x = train_x / 255.0
        testval_x = testval_x / 255.0
    
    if onehot:
        num_classes=10
        train_y=keras.utils.to_categorical(train_y,num_classes)
        testval_y=keras.utils.to_categorical(testval_y,num_classes)
    # split test - val
    nval = testval_x.shape[0] // 2
    
    val_x = testval_x[:nval]
    val_y = testval_y[:nval]
    
    test_x = testval_x[nval:]
    test_y = testval_y[nval:]
    
    
#     val_x,val_y,test_x,test_y=train_test_split(test_images,test_labels,train_size=int(test_images.shape[0]/2), test_size=int(test_images.shape[0]/2))

    
    
    return train_x, train_y, test_x, test_y, val_x, val_y, labels

In [134]:
# do some sanity checks
train_x, train_y, test_x, test_y, val_x, val_y, labels = load_preprocess_cifar10(onehot=False, minmax_normalize=False)
assert train_x.shape[0] == train_y.shape[0] and test_x.shape[0] == test_y.shape[0] and val_x.shape[0] == val_y.shape[0]
assert np.amax(train_x) >= 255 and np.amax(test_x) >= 255 and np.amax(val_x) >= 255
assert train_y.shape == (train_y.shape[0],) or train_y.shape == (train_y.shape[0],1)

train_x, train_y, test_x, test_y, val_x, val_y, labels = load_preprocess_cifar10(onehot=True, minmax_normalize=False)
assert np.amax(train_x) >= 255 and np.amax(test_x) >= 255 and np.amax(val_x) >= 255
assert train_y.shape == (train_y.shape[0],10) and train_y.shape[1] == test_y.shape[1]

In [135]:
# actually load the data
train_x, train_y, test_x, test_y, val_x, val_y, labels = load_preprocess_cifar10()
assert np.amax(train_x) <= 1 and np.amax(test_x) <= 1 and np.amax(val_x) <= 1
assert np.amax(train_x) >= 0 and np.amax(test_x) >= 0 and np.amax(val_x) >= 0

assert labels.shape[0] == 10 and labels.shape[0] == train_y.shape[1]

## [Task 2] (30 points) Training a CNN for Cifar-10

#### We will use the following architecture
- Conv layer with 32 filters, (3,3) filter size, stride of 1, padding 'same'
- Conv layer with 32 filters, (3,3) filter size, stride of 1, padding 'same'
- Max pooling layer (2,2)
- Dropout with rate 25%
- Conv layer with 64 filters, (3,3) filter size, stride of 1, padding 'same'
- Conv layer with 64 filters, (3,3) filter size, stride of 1, padding 'same'
- Max pooling layer (2,2)
- Dropout with rate 25%
- Conv layer with 128 filters, (3,3) filter size, stride of 1, padding 'same'
- Conv layer with 128 filters, (3,3) filter size, stride of 1, padding 'same'
- Max pooling layer (2,2)
- Dropout with rate 25%
- Flatten
- FC with 128 units
- Dropout with rate 25%
- FC with 64 units
- Dropout with rate 25%
- (Output layer) FC with 10 units

#### For all layers (if applicable) except the output layer you should use:
- ReLU as activation function
- He uniform weight initialization strategy
- L2 regularization with regularization constant set to 0.001

#### For the output layer you should select a suitable activation function that is consistent with the task and loss function you use. Use Adam for the optimizer with learning rate 0.002.

### [Task 2a] (20 points) Implement create_compile_cnn() according to the architecture specified above.

In [136]:
from keras.regularizers import l2

def create_compile_cnn(input_shape=[32, 32, 3], num_outputs=10, verbose=False):
    
    model = keras.models.Sequential(name='CIFAR-10--CNN')
    initializer = tf.keras.initializers.HeUniform()
    ### Don't forget to compile the model and print the summary if verbose=True
    ###* put your code here (~20 lines) *###
    model.add(Conv2D(filters=32, input_shape=input_shape, kernel_size=(3,3), strides=(1,1), padding='same',kernel_initializer=initializer,kernel_regularizer=l2(0.001)))
    model.add(Activation('relu'))
    model.add(Conv2D(filters=32,kernel_size=(3,3), strides=(1,1), padding='same',kernel_initializer=initializer,kernel_regularizer=l2(0.001)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same',kernel_initializer=initializer,kernel_regularizer=l2(0.001)))
    model.add(Activation('relu'))
    model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same',kernel_initializer=initializer,kernel_regularizer=l2(0.001)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same',kernel_initializer=initializer,kernel_regularizer=l2(0.001)))
    model.add(Activation('relu'))
    model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same',kernel_initializer=initializer,kernel_regularizer=l2(0.001)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    
    model.add(Dense(128,kernel_initializer=initializer,kernel_regularizer=l2(0.001)))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    
    model.add(Dense(64,kernel_initializer=initializer,kernel_regularizer=l2(0.001)))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    
    model.add(Dense(10))
    model.add(Activation('softmax'))

    opt=keras.optimizers.Adam(lr=0.002)   
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

In [137]:
_ = create_compile_cnn(verbose=True)

### [Task 2b] (10 points) Train the model. Fill in the implementation below.

In [138]:
cnn_model_fp = './cifar10-cnn.h5'

# If the model file exists, load it. Otherwise train it and save the model.
# Note: if you need to retrain the model, simply delete the h5 file.
if os.path.exists(cnn_model_fp):
    model = keras.models.load_model(cnn_model_fp)
else:
    model = create_compile_cnn(verbose=False)
    
    # train the model using model.fit() for at least 3 epochs and your chosen batch_size
    # you can set any callback you want on it, including checkpoint, early stopping, etc.
    ###* put your code here (~3-5 lines) *###
    early_stop_cb = EarlyStopping(monitor='val_accuracy', patience=3)
    checkpoint_cb = ModelCheckpoint(cnn_model_fp, monitor='val_accuracy', save_best_only=True, mode='max')
    max_epochs=15
    batch_size=128
    model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=max_epochs, batch_size=batch_size, shuffle=True, callbacks=[early_stop_cb,checkpoint_cb])
    
    
    # save the model
    model.save(cnn_model_fp)

Epoch 1/15
391/391 [==============================] - 161s 409ms/step - loss: 2.9364 - accuracy: 0.1872 - val_loss: 1.9072 - val_accuracy: 0.3922
Epoch 2/15
391/391 [==============================] - 158s 404ms/step - loss: 1.8720 - accuracy: 0.3782 - val_loss: 1.5613 - val_accuracy: 0.4890
Epoch 3/15
391/391 [==============================] - 164s 419ms/step - loss: 1.6365 - accuracy: 0.4682 - val_loss: 1.4081 - val_accuracy: 0.5536
Epoch 4/15
391/391 [==============================] - 155s 396ms/step - loss: 1.4982 - accuracy: 0.5263 - val_loss: 1.3842 - val_accuracy: 0.5656
Epoch 5/15
391/391 [==============================] - 164s 419ms/step - loss: 1.4241 - accuracy: 0.5624 - val_loss: 1.2342 - val_accuracy: 0.6330
Epoch 6/15
391/391 [==============================] - 164s 420ms/step - loss: 1.3608 - accuracy: 0.5878 - val_loss: 1.1739 - val_accuracy: 0.6606
Epoch 7/15
391/391 [==============================] - 157s 403ms/step - loss: 1.3207 - accuracy: 0.6080 - val_loss: 1.1892 -

In [139]:
# let's evaluate the model on the test data
loss, acc = model.evaluate(test_x, test_y, verbose=0)
print('[Model] Test accuracy: {:.2f}%'.format(100*acc))

[Model] Test accuracy: 72.06%


## [Task 3] (15 points) Processing Sequence Data

### [Task 3a] (15 points) Fill in the implementation of load_preprocess_imdb()

In [140]:
from tensorflow.keras.datasets import imdb

# the size of the vocabulary we'll use
vocab_size = 12000
maxlen = 150

# refer to: https://www.tensorflow.org/api_docs/python/tf/keras/datasets/cifar10/load_data
# and to https://www.cs.toronto.edu/~kriz/cifar.html
def load_preprocess_imdb(num_words=vocab_size, prop_vec=prop_vec, maxlen=maxlen, vectorize=False):
    
    np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
    
    train, testval = imdb.load_data(num_words=num_words, maxlen=maxlen, oov_char=0)
    
    np.warnings.filterwarnings('default', category=np.VisibleDeprecationWarning)    
    
    ### Process the data 
    ### Merge train and testval, but then split again into train, test, val sets (according to prop_vec). You can use utils.train_test_val_split().)
    ### - If vectorize=True, then you must encode the features of each example into vectors of vocab_size entries 
    ### such that entry i contains the number of time word i appeared in the sequence
    ### - If vectorize=False, then you must encode the features of each examples as a sequence of size maxlen (represented as a np.array()). 
    ### Make sure to pad sequences with 0 as appropriate.
    ###* put your code here (~10-15 lines) *###
    x=np.concatenate((train[0],testval[0]))
    y=np.concatenate((train[1],testval[1]))
    if vectorize:
        new_x=np.zeros((len(x),vocab_size))
        for (i,j) in enumerate(x):
            for word_num in j:
                new_x[i,word_num]+=1
        x=new_x
    else:
        x=tf.keras.preprocessing.sequence.pad_sequences(x,maxlen=maxlen)
    
    train_x, train_y, test_x, test_y, val_x, val_y = utils.train_test_val_split(x, y, prop_vec, shuffle=True, seed=seed)

    return train_x, train_y, test_x, test_y, val_x, val_y

In [141]:
# sanity checks
train_x, train_y, test_x, test_y, val_x, val_y = load_preprocess_imdb(vectorize=False)
assert train_x.shape == (16281, maxlen) and train_y.shape == (train_x.shape[0],)

train_x, train_y, test_x, test_y, val_x, val_y = load_preprocess_imdb(vectorize=True)
assert train_x.shape == (16281, vocab_size) and train_y.shape == (train_x.shape[0],)

## [Task 4] (35 points) RNN for Sentiment Analysis

### [Task 4a] (35 points) Complete the code below to define an RNN architecture for sentiment analysis. The goal is to predict the sentiment of IMDB reviews. You can use any architecture you want, but a good place to start would be to use an embedding layer followed by some recurrent layers (e.g., LSTM, GRU, etc.). Keep the number of parameters of the model below 2m.

In [142]:
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
def create_compile_rnn(input_shape=[None], vocab_size=vocab_size, embedding_size=128, num_outputs=1, verbose=False):
    
    model = keras.models.Sequential(name='imdb-RNN')
    
    ### Don't forget to compile the model and print the summary if verbose=True
    ### Use binary_crossentropy as loss function.    
    ###* put your code here (~15-20 lines) *###

    model.add(Embedding(vocab_size, embedding_size, input_length=150))
    model.add(LSTM(100))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])

    if(verbose):
        model.summary()
    
    
    return model

In [143]:
model = create_compile_rnn(verbose=True)

Model: "imdb-RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 150, 128)          1536000   
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               91600     
_________________________________________________________________
dense_58 (Dense)             (None, 1)                 101       
Total params: 1,627,701
Trainable params: 1,627,701
Non-trainable params: 0
_________________________________________________________________


In [144]:
# let's load the data
train_x, train_y, test_x, test_y, val_x, val_y = load_preprocess_imdb(vectorize=False)


early_stop_cb = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# feel free to tweak the batch size, number of epochs and callbacks.
max_epochs = 3
batch_size = 32

hist = model.fit(train_x, train_y, epochs=max_epochs, batch_size=batch_size, validation_data=(val_x, val_y), callbacks=[early_stop_cb])

Epoch 1/3
509/509 [==============================] - 59s 112ms/step - loss: 0.5690 - accuracy: 0.6805 - val_loss: 0.3201 - val_accuracy: 0.8702
Epoch 2/3
509/509 [==============================] - 54s 105ms/step - loss: 0.2187 - accuracy: 0.9216 - val_loss: 0.3221 - val_accuracy: 0.8761
Epoch 3/3
509/509 [==============================] - 59s 116ms/step - loss: 0.1304 - accuracy: 0.9556 - val_loss: 0.3581 - val_accuracy: 0.8695


In [145]:
# let's evaluate the model on the test data
loss, acc = model.evaluate(test_x, test_y, verbose=0)
print('[Model] Test accuracy: {:.2f}%'.format(100*acc))

[Model] Test accuracy: 88.43%


## [CIS6930 Additional Task -- Task 5] (25 points): DNN for Sentiment Analysis

### In the previous task, we use an RNN for sentiment analysis. In this task you will use a neural network without any recurrent layers for the same task as a comparison.

### We'll use the data in vectorized form for this.

### [Task 5a] (20 points) Complete the code below to define an architecture of your choice *without* any recurrent layers. The goal is to get the best model with the fewest number of parameters. Keep the number of parameters of the model below 2m and ideally similar to the model of Task 4.

In [146]:
def create_compile_dnn(input_shape=[vocab_size], num_outputs=1, verbose=False):
    
    model = keras.models.Sequential(name='imdb-DNN')
    
    ### Don't forget to compile the model and print the summary if verbose=True
    ###* put your code here (~10 lines) *###

    model.add(keras.Input(shape=input_shape, sparse=False, name='input')) 
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))


    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    if(verbose):
        model.summary()
    return model

In [147]:
model = create_compile_dnn(verbose=True)

Model: "imdb-DNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_59 (Dense)             (None, 128)               1536128   
_________________________________________________________________
dropout_27 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_60 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_28 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_61 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_62 (Dense)             (None, 1)                 33        
Total params: 1,546,497
Trainable params: 1,546,497
Non-trainable params: 0
________________________________________________

In [148]:
# Let's load the data in vectorized form
train_x, train_y, test_x, test_y, val_x, val_y = load_preprocess_imdb(vectorize=True)


early_stop_cb = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# feel free to tweak the batch size, number of epochs and callbacks.
max_epochs = 50
batch_size = 100

hist = model.fit(train_x, train_y, epochs=max_epochs, batch_size=batch_size,validation_data=(val_x, val_y), callbacks=[early_stop_cb])

Epoch 1/50
163/163 [==============================] - 7s 35ms/step - loss: 0.4858 - accuracy: 0.7535 - val_loss: 0.2795 - val_accuracy: 0.8894
Epoch 2/50
163/163 [==============================] - 3s 17ms/step - loss: 0.1685 - accuracy: 0.9384 - val_loss: 0.2976 - val_accuracy: 0.8761
Epoch 3/50
163/163 [==============================] - 3s 16ms/step - loss: 0.0834 - accuracy: 0.9730 - val_loss: 0.3513 - val_accuracy: 0.8776
Epoch 4/50
163/163 [==============================] - 3s 17ms/step - loss: 0.0377 - accuracy: 0.9885 - val_loss: 0.4551 - val_accuracy: 0.8754
Epoch 5/50
163/163 [==============================] - 3s 16ms/step - loss: 0.0229 - accuracy: 0.9927 - val_loss: 0.6347 - val_accuracy: 0.8650
Epoch 6/50
163/163 [==============================] - 3s 16ms/step - loss: 0.0179 - accuracy: 0.9938 - val_loss: 0.6002 - val_accuracy: 0.8650


In [149]:
# let's evaluate the model on the test data
loss, acc = model.evaluate(test_x, test_y, verbose=0)
print('[Model] Test accuracy: {:.2f}%'.format(100*acc))

[Model] Test accuracy: 89.98%


### [Task 5b] (5 points) Compare this model to the model of Task 4. What do you conclude?

In [150]:
###* put your answer here *###
# I am able to get similar accuracy to the model of Task 4. But, I think that the model is over fitting as the validation 
#  loss for my dnn model was increasing.
#